# Baseline Features

The baseline features include: `Timestamp`, `Open`, `High`, `Low`, `Close`, `Volume`.
- Working with the full index dataset (7.0M entries) can lead to excessive training times.
- We'll restrict our dataset to the most recent `100,000` data points, which are likely more relevant for future predictions.
- Note that a `training period` of only two months may not capture all relevant trends, despite containing substantial data.

In [ ]:
import os
import pandas as pd

PROCESSED_PATH = "data/processed/"
PROCESSED_NAME = "btcusd_1-min_data_processed.csv"
PROCESSED_FILE = os.path.join(PROCESSED_PATH, PROCESSED_NAME)

df = pd.read_csv(PROCESSED_FILE)
df["Timestamp"] = pd.to_datetime(df["Timestamp"], unit="s", utc=True)
df.set_index("Timestamp", inplace=True)
df.drop(columns=["datetime"], inplace=True)
df.info()

## Data Splitting Strategy

For our time series, we'll split the dataset into three distinct segments:

1. **Training Set (60%)**: The earliest portion of our chronological data used to train the model and establish patterns.

2. **Validation Set (20%)**: The middle segment used to tune hyperparameters and prevent overfitting during the development phase.

3. **Test Set (20%)**: The most recent data, kept completely separate until final evaluation to simulate real-world performance.

This chronological splitting approach is crucial for financial time series data to prevent data leakage and maintain the temporal nature of the information.

In [ ]:
# Chronological split with 60/20/20
train_size = int(0.6 * len(df))
valid_size = int(0.2 * len(df))

# Split in chronological order
train_data = df.iloc[:train_size]
valid_data = df.iloc[train_size : train_size + valid_size]
test_data = df.iloc[train_size + valid_size :]

print(f"Training set: {len(train_data)} entries")
print(f"Validation set: {len(valid_data)} entries")
print(f"Test set: {len(test_data)} entries")

# Check time ranges
print(f"Training set: {train_data.index.min()} to {train_data.index.max()}")
print(f"Validation set: {valid_data.index.min()} to {valid_data.index.max()}")
print(f"Test set: {test_data.index.min()} to {test_data.index.max()}")

# Saving the split data records
train_data.to_csv(os.path.join(PROCESSED_PATH, "train_data.csv"))
valid_data.to_csv(os.path.join(PROCESSED_PATH, "validation_data.csv"))
test_data.to_csv(os.path.join(PROCESSED_PATH, "test_data.csv"))

# TARGET VARIABLE

- We define our prediction variable `signal` using the `Close` price relative to the Bull Market Support Band (the lower of 20-week SMA and 21-week EMA).
- If closing price falls below the Bull Market Support Band, it signals a bear market, otherwise a bull market.
- The trading strategy assigns signal value = 1 (buy) in bull markets and signal value = 0 (sell) in bear markets.
- The window values for both moving averages are configurable parameters, both of which are arbitrary, and can affect the results, ideally an optimisation study needs to be carried out to find optimum values.


In [ ]:
import numpy as np


def generate_bull_bear_signals(price_data, verbose=True):
    # Calculate 20-week SMA for minute-level data
    # 20 weeks = 20 weeks * 7 days * 24 hours * 60 minutes = 201,600 minutes
    price_data["SMA_20W"] = (
        price_data["Close"].rolling(window=201600, min_periods=1, center=False).mean()
    )

    # Calculate 21-week EMA for minute-level data
    # 21 weeks = 21 weeks * 7 days * 24 hours * 60 minutes = 211,680 minutes
    price_data["EMA_21W"] = (
        price_data["Close"].ewm(span=211680, min_periods=1, adjust=False).mean()
    )

    # Create Bull Market Support Band (the lower of the two indicators)
    price_data["Bull_Support_Band"] = price_data[["SMA_20W", "EMA_21W"]].min(axis=1)

    # Create signals: 1 (buy) when Close is above Bull Market Support Band, 0 (sell) otherwise
    price_data["signal"] = np.where(
        price_data["Close"] > price_data["Bull_Support_Band"], 1.0, 0.0
    )

    if verbose:
        display(price_data["signal"].value_counts())

    return price_data


train_data_with_signals = train_data.copy()
valid_data_with_signals = valid_data.copy()

generate_bull_bear_signals(train_data_with_signals, verbose=True)
generate_bull_bear_signals(valid_data_with_signals, verbose=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


def plot_target_correlation(df, target="demand", figsize=(12, 1), return_corr=False):
    # Calculate correlation matrix
    corr = df.corr()[target].drop(target, errors="ignore")

    if return_corr:
        return corr

    # Plot
    plt.figure(figsize=figsize)
    sns.heatmap(
        corr.to_frame().T,
        annot=True,
        cmap="coolwarm",
        center=0,
        vmin=-0.3,
        vmax=0.3,
        cbar=False,
        linewidths=1,
    )
    plt.title(f"Feature Correlation with {target}")
    plt.tight_layout()

    return plt.gcf()

In [ ]:
plot_target_correlation(train_data_with_signals, "signal", figsize=(7, 0.5))